In [55]:
# http://yann.lecun.com/exdb/mnist/
# https://medium.com/@mannasiladittya/converting-mnist-data-in-idx-format-to-python-numpy-array-5cb9126f99f1
# https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40

# Deskewing https://fsix.github.io/mnist/Deskewing.html
from PIL import Image
import struct as st
import numpy as np
filename = {'images' : 'data/train-images-idx3-ubyte' ,'labels' : 'data/train-labels-idx1-ubyte'}

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = st.unpack('>HBB', f.read(4))
        shape = tuple(st.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [56]:
from scipy.ndimage import interpolation

def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

In [65]:
train_size = 1000
labels = read_idx(filename['labels'])
labels = labels[:train_size]

In [70]:
len(matrix)

60000

In [72]:
images = []
deskewed_images = []

image_matrices = []
deskewed_image_matrices = []

matrix = read_idx(filename['images'])


for a in range(train_size):
    image_matrices.append(matrix[a])    
    deskewed_image_matrices.append(deskew(matrix[a]))

    images.append(Image.fromarray(matrix[a]))
    deskewed_images.append(Image.fromarray(deskew(matrix[a])))

In [74]:
matrix.shape

(60000, 28, 28)

In [75]:
counter = 0
#for image, deskewed_image, label, matrix in zip(images, deskewed_images, labels, image_matrices):
for image, label, matrix in zip(images, labels, image_matrices):
    counter +=1
    if counter >5: break
    display(image, label)#, matrix)

5

0

4

1

9

In [76]:
X = np.array([])
counter = 0
#for matrix in deskewed_image_matrices:
#manual unrolling, unnecessary with keras.layers.Flatten

for matrix in image_matrices[:train_size]:
    counter +=1
    if not counter % 1000: print(counter)
    unrolled = np.array([])
    for row_index in range(matrix.shape[0]):
        if not unrolled.size>0:
            unrolled = matrix[row_index]
        else:
            unrolled = np.hstack((unrolled, matrix[row_index]))
    unrolled = unrolled.reshape(-1,1)
    if not X.size>0: X = unrolled
    else: X = np.hstack((X, unrolled))
print(X.shape)

1000
(784, 1000)


In [77]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels)

ValueError: Found input variables with inconsistent numbers of samples: [784, 1000]

In [89]:
#y_train = y_train.reshape(-1,)

In [32]:
lm = LogisticRegression(solver='liblinear', multi_class='auto')
#lm = RidgeClassifier(alpha=1)
lm.fit(X_train, y_train)
score = lm.score(X_test, y_test)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [33]:
score

0.8472